In [ ]:
# Data Crawler
# gopax 거래소에서 받고싶은 정보의 시간 범위를 작성하면 종가를 csv파일로 저장해주는 프로그램

In [1]:
import gopax_api_wrapper as gopax
import numpy as np
import pandas as pd
from datetime import datetime

In [2]:
#data frame을 받기
def get_past_dataframe(trading_pair, start, end, interval):
    start_ts = gopax.get_timestamp(start)
    end_ts = gopax.get_timestamp(end)
    raw_result = gopax.get_past_data(trading_pair, start_ts, end_ts, interval)
    result = pd.DataFrame(raw_result, columns = ['time', 'low', 'high', 'open', 'close', 'volume'])
    return result

In [3]:
#data frame 시간 조정
def get_clean_data(trading_pair, start, end, interval):
    diff = 18 * 3600
    s = start
    e = end
    ts = gopax.get_timestamp(s)
    te = gopax.get_timestamp(e)
    result = get_past_dataframe(trading_pair, s, e, interval)
    result = result[result.time >= ts * 1000]
    result = result[result.time < te * 1000]
    result['time'] = result['time'].apply(lambda x: datetime.utcfromtimestamp((x // 1000) + diff))
    return result

In [28]:
#시작 시간 지정
#UTC시간으로 변경시키는 과정

from time import mktime, strptime
time_format = '%Y-%m-%d %H:%M:%S'
time_tuple = strptime('2018-10-31 00:00:00', time_format)
utc_start = mktime(time_tuple)

#마지막 시간 지정
#UTC시간으로 변경시키는 과정

time_format = '%Y-%m-%d %H:%M:%S'
time_tuple = strptime('2018-11-10 00:00:00', time_format)
utc_end = mktime(time_tuple)

In [29]:
#거래소에서 불러올 시간 간격 설정

start_utc_time = utc_start      #2018-01-01 10:00:00 시작
start_end_term = 60*60*24    #시작-마지막 간격 = 1일
next_data_term = 3600         #train 데이터간 시간 간격 = 1시간
time_term = 30                 #데이터간 시간 간격 = 30분

label_time_term = 60*90   #lable 데이터 시작 마지막 간격

slack = 60*60*4           #데이터 여유분(고팍스 데이터 불러오는거 그지 같아서...)
end_utc_time = utc_end
how_long = int((utc_end - utc_start) / (next_data_term))

In [30]:
# how_long에 저장할 개수를 지정
#df_train 딕셔너리에 df_train_0 이런 식으로 데이터 저장하기
#18년 1월 1일 부터 시작해서 일주일간의 데이터 적용

df_train = {}
value_sum = np.array(())

for how_long in range(how_long):
    df_train["df_train_"+str(how_long)] = \
    get_clean_data('BTC-KRW', str(datetime.utcfromtimestamp(start_utc_time+how_long*next_data_term)), 
                       str(datetime.utcfromtimestamp(start_utc_time+start_end_term+how_long*next_data_term+label_time_term+slack)),
                   time_term)
    
    if how_long == 0:
        value = df_train["df_train_"+str(0)]["close"].values[:51]
        print(np.shape(value))        
    else:
        value= np.vstack([value, df_train["df_train_"+str(how_long)]["close"].values[:51]]) 

        #train데이터를 exel로 옮기기
a = np.asarray(value)
print(np.shape(a))
np.savetxt("train_24.csv", a, delimiter=",")
print(df_train['df_train_0'][:51])

(51,)
(240, 51)
                  time      low     high     open    close     volume
2  2018-10-31 00:00:00  7145000  7152000  7150000  7152000   4.140182
3  2018-10-31 00:30:00  7145000  7153000  7152000  7153000   5.186351
4  2018-10-31 01:00:00  7153000  7155000  7153000  7155000   8.392444
5  2018-10-31 01:30:00  7150000  7166000  7155000  7166000   4.759775
6  2018-10-31 02:00:00  7145000  7165000  7165000  7145000   3.487650
7  2018-10-31 02:30:00  7142000  7156000  7155000  7154000  16.362149
8  2018-10-31 03:00:00  7140000  7158000  7144000  7148000  10.647679
9  2018-10-31 03:30:00  7141000  7159000  7148000  7155000   4.376992
10 2018-10-31 04:00:00  7148000  7161000  7152000  7151000   4.147805
11 2018-10-31 04:30:00  7151000  7159000  7159000  7152000   2.882003
12 2018-10-31 05:00:00  7151000  7155000  7151000  7152000   1.220740
13 2018-10-31 05:30:00  7147000  7155000  7147000  7155000   5.062800
14 2018-10-31 06:00:00  7154000  7164000  7155000  7159000  10.178583
15 2